In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [2]:
df = pd.read_csv("/Users/pranavi/Downloads/HeartAttack.csv", na_values="?")

In [3]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,28,1,2,130.0,132.0,0.0,2.0,185.0,0.0,0.0,NaN,NaN,NaN,0
1,29,1,2,120.0,243.0,0.0,0.0,160.0,0.0,0.0,NaN,NaN,NaN,0
2,29,1,2,140.0,NaN,0.0,0.0,170.0,0.0,0.0,NaN,NaN,NaN,0
3,30,0,1,170.0,237.0,0.0,1.0,170.0,0.0,0.0,NaN,NaN,6.0,0
4,31,0,2,100.0,219.0,0.0,1.0,150.0,0.0,0.0,NaN,NaN,NaN,0


In [4]:
df.isnull().sum()

age             0
sex             0
cp              0
trestbps        1
chol           23
fbs             8
restecg         1
thalach         1
exang           1
oldpeak         0
slope         190
ca            291
thal          266
num             0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         294 non-null    int64  
 1   sex         294 non-null    int64  
 2   cp          294 non-null    int64  
 3   trestbps    293 non-null    float64
 4   chol        271 non-null    float64
 5   fbs         286 non-null    float64
 6   restecg     293 non-null    float64
 7   thalach     293 non-null    float64
 8   exang       293 non-null    float64
 9   oldpeak     294 non-null    float64
 10  slope       104 non-null    float64
 11  ca          3 non-null      float64
 12  thal        28 non-null     float64
 13  num         294 non-null    int64  
dtypes: float64(10), int64(4)
memory usage: 32.3 KB


In [6]:
df.drop(["ca","slope","thal"], axis=1, inplace=True)

In [7]:
df.dropna(inplace=True)

In [8]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,num
0,28,1,2,130.0,132.0,0.0,2.0,185.0,0.0,0.0,0
1,29,1,2,120.0,243.0,0.0,0.0,160.0,0.0,0.0,0
3,30,0,1,170.0,237.0,0.0,1.0,170.0,0.0,0.0,0
4,31,0,2,100.0,219.0,0.0,1.0,150.0,0.0,0.0,0
5,32,0,2,105.0,198.0,0.0,0.0,165.0,0.0,0.0,0


In [9]:
df.isnull().sum()

age           0
sex           0
cp            0
trestbps      0
chol          0
fbs           0
restecg       0
thalach       0
exang         0
oldpeak       0
num           0
dtype: int64

In [10]:
x = df.iloc[:,:-1]
y = df["num       "]

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [12]:
sc = StandardScaler()

In [13]:
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [14]:
x_train.shape

(208, 10)

In [15]:
x_test.shape

(53, 10)

In [16]:
x_train = torch.tensor(x_train).float()
x_test = torch.tensor(x_test).float()
y_train = torch.tensor(y_train.values).long()
y_test = torch.tensor(y_test.values).long()

In [17]:
class ANN(nn.Module):
    def __init__(self):
        super(ANN,self).__init__()
        
        self.fc1 = nn.Linear(10, 100)
        self.fc2 = nn.Linear(100, 25)
        self.fc3 = nn.Linear(25,2)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        out = self.relu(self.fc1(x))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        
        return out
        

In [18]:
model = ANN()
print(model.parameters)

<bound method Module.parameters of ANN(
  (fc1): Linear(in_features=10, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=25, bias=True)
  (fc3): Linear(in_features=25, out_features=2, bias=True)
  (relu): ReLU()
)>


In [33]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [36]:
epochs =10

model.train()

for epoch in range(epochs):
    train_acc = 0.0
    train_loss = 0.0
    
    optimizer.zero_grad() 
    output = model(x_train)
    
    predictions = torch.argmax(output,1)
    
    train_acc += (predictions == y_train).sum().item()
    loss = loss_fn(output, y_train)
    
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    
    print("Training Acc:{:.2f}, Training Loss:{:.2f}".format(train_acc/len(x_train), train_loss))

Training Acc:0.88, Training Loss:0.26
Training Acc:0.88, Training Loss:0.25
Training Acc:0.89, Training Loss:0.24
Training Acc:0.90, Training Loss:0.23
Training Acc:0.89, Training Loss:0.22
Training Acc:0.92, Training Loss:0.21
Training Acc:0.92, Training Loss:0.20
Training Acc:0.92, Training Loss:0.19
Training Acc:0.93, Training Loss:0.19
Training Acc:0.93, Training Loss:0.18


In [37]:
pred = model(x_test)

In [38]:
print(pred)

tensor([[-3.2710,  1.9989],
        [ 1.2890, -2.0011],
        [ 2.4547, -3.3319],
        [ 3.5011, -5.1304],
        [ 1.4240, -2.8543],
        [-0.5424, -0.1667],
        [ 3.1437, -4.4108],
        [ 1.0395, -1.9141],
        [-0.1095, -0.7579],
        [-2.3664,  1.6989],
        [-1.6413,  1.0184],
        [ 1.1531, -2.2642],
        [ 0.4664, -1.3265],
        [ 0.1638, -1.0241],
        [ 0.1746, -0.9085],
        [-1.6499,  0.9791],
        [-0.1987, -0.1133],
        [-0.6534, -0.2704],
        [-0.1393, -0.7479],
        [ 0.8361, -1.7199],
        [ 2.9341, -4.6529],
        [-1.6450,  0.9282],
        [-1.2200,  0.0747],
        [ 3.5231, -5.2585],
        [ 2.1092, -3.1160],
        [ 0.2463, -0.8800],
        [ 2.0418, -3.4334],
        [ 1.5009, -2.5975],
        [-3.5115,  2.4122],
        [-0.4613, -0.1202],
        [ 1.3665, -2.1320],
        [ 0.9640, -2.2012],
        [ 0.0932, -0.5151],
        [ 3.4919, -4.4971],
        [-0.6261,  0.0073],
        [-2.2175,  1

In [41]:
pred_index = torch.argmax(pred,1)

In [42]:
pred_index

tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        1, 0, 0, 0, 1])

In [43]:
from sklearn.metrics import accuracy_score

In [44]:
print(accuracy_score(pred_index, y_test))

0.7924528301886793
